In [1]:
from platform import python_version
print(python_version())

3.11.0


# typing — Support for type hints

In [6]:
def greeting(name: str) -> str:
    return 'Hello ' + name
display(greeting("types!"))

'Hello types!'

## Type aliases

In [7]:
Vector = list[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

# passes type checking; a list of floats qualifies as a Vector.
new_vector = scale(2.0, [1.0, -4.2, 5.4])
display(new_vector)

[2.0, -8.4, 10.8]

In [9]:
from collections.abc import Sequence

ConnectionOptions = dict[str, str]
Address = tuple[str, int]
Server = tuple[Address, ConnectionOptions]

def broadcast_message(message: str, servers: Sequence[Server]) -> None:
    ...

# The static type checker will treat the previous type signature as
# being exactly equivalent to this one.
def broadcast_message(
        message: str,
        servers: Sequence[tuple[tuple[str, int], dict[str, str]]]) -> None:
    ...

## NewType

In [16]:
from typing import NewType

UserId = NewType('UserId', int)
some_id = UserId(524313)

def get_user_name(user_id: UserId) -> str:
    ...

# passes type checking
user_a = get_user_name(UserId(42351))

# fails type checking; an int is not a UserId
user_b = get_user_name(-1)

# 'output' is of base type 'int', not 'UserId'
output = UserId(23413) + UserId(54341)

# Fails at runtime and does not pass type checking - It is invalid to create a subtype of derived:
# class AdminUserId(UserId): pass

# it is possible to create a NewType based on a ‘derived’ NewType:
ProUserId = NewType('ProUserId', UserId)

## Callable

In [22]:
from collections.abc import Callable, Awaitable

def feeder(get_next_item: Callable[[], str]) -> None:
    ...

def async_query(on_success: Callable[[int], None],
                on_error: Callable[[int, Exception], None]) -> None:
    ...

async def on_update(value: str) -> None:
    ...
    
callback: Callable[[str], Awaitable[None]] = on_update

## Generics

In [29]:
from collections.abc import Mapping, Sequence

class Employee(): pass

def notify_by_email(employees: Sequence[Employee], 
                    overrides: Mapping[str, str]) -> None:  ...

from typing import TypeVar

T = TypeVar('T')      # Declare type variable

def first(l: Sequence[T]) -> T:   # Generic function
    return l[0]

## User-defined generic types

In [41]:
from typing import TypeVar, Generic
from logging import Logger

T = TypeVar('T')

class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str, logger: Logger) -> None:
        self.name = name
        self.logger = logger
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        self.logger.warning('%s: %s', self.name, message)
        
import logging
logger = logging.getLogger('example_logger')
logger.info('This is an info')
logger.warning('This is an warning')

l = LoggedVar("hello ", "blah+logger", logger)
l.log("blah")
l.set("echo")
print(l.get())

This is an warning
blah+logger: blah
blah+logger: Set 'hello '
blah+logger: Get 'echo'


echo


In [42]:
from collections.abc import Iterable

def zero_all_vars(vars: Iterable[LoggedVar[int]]) -> None:
    for var in vars:
        var.set(0)

In [43]:
from typing import TypeVar, Generic, Sequence

T = TypeVar('T', contravariant=True)
B = TypeVar('B', bound=Sequence[bytes], covariant=True)
S = TypeVar('S', int, str)

class WeirdTrio(Generic[T, B, S]):
    ...

In [2]:
from collections.abc import Iterable
from typing import TypeVar
S = TypeVar('S')

Response = Iterable[S] | int

# Return type here is same as Iterable[str] | int
def response(query: str) -> Response[str]:
    ...

T = TypeVar('T', int, float, complex)
Vec = Iterable[tuple[T, T]]

def inproduct(v: Vec[T]) -> T: # Same as Iterable[tuple[T, T]]
    return sum(x*y for x, y in v)